# Send Emails with Data from Excel

The purpose of this example is to demonstrate how to send HTML emails which include data from an Excel worksheet. Sending the email as an HTML email allows for data to be included in the body of the email in an HTML table.

## Import needed modules

In [ ]:
# This module may need to be installed before it can be used.
# https://dev.to/siddheshshankar/convert-a-dataframe-into-a-pretty-html-table-and-send-it-over-email-4663
# !pip install pretty-html-table

In [46]:
import smtplib
from email.message import EmailMessage
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
from pretty_html_table import build_table

## Some setup needed

There are some settings which need to be made depending on what mail server you're going to use. The location where I first developed this solution had an SMTP server available, so the setting in this example are fairly simple. Other mail solutions may require additional configuration.

In [13]:
email_host = 'smtp.xx.xx.com' # This is an example. You'll need to find the correct address for the email host you'll use.
port = 25 # This is a common setting for an smtp server.


## Main function to send mail

In [ ]:
def send_message2(fromadd, to, cc, bcc, subject, salutation, body, signature, note=None, table=None):
    message_text = '<p>' + salutation + '</p>\n<p>' + body
    if note:
        message_text += '</p>\n<p>' + note
    if table:
        message_text += '</p>\n<p>' + table
    message_text += '</p>\n<p>' + signature + '</p>'
    msg = MIMEMultipart()
    msg['From'] = fromadd
    msg['To'] = to
    mail_add = msg['To']
    if cc:
        msg['Cc'] = cc
        mail_add += ',' + cc
    if bcc:
        msg['Bcc'] = bcc
        mail_add += ',' + bcc
    msg['Subject'] = subject
    msg.attach(MIMEText(message_text, 'html'))
    
    mail_list = mail_add.split(',')
    with smtplib.SMTP(host=email_host, port=port) as smtp:
        smtp.sendmail(msg['From'], mail_list, msg.as_string())
        


## Open the Excel file with all the needed information into a Pandas dataframe

I also filter the rows in the dataframe which I have marked with a status of "ready" so I only send emails I intend to send right now, but keep older rows for reference.

In [ ]:
path = "IntakeList.xlsx"
df = pd.read_excel(path, sheet_name="emails", dtype=object)
df = df.fillna('')
df = df[(df['status'] == 'ready')]

## Apply the send_message2 function to each row of the dataframe.

This generates an email from each row in the dataframe and sends it using the SMTP server.

In [1]:

df.apply(lambda row : send_message2(row['from'], row['to'], row['cc'], row['bcc'], row['subject'], row['salutation'], row['body'], row['signature'], build_table(pd.DataFrame({'Notes':[row['Notes']]}), 'blue_light', font_size='small', width='800px'), build_table(pd.DataFrame({k:[row[k]] for k in res}), 'blue_light', font_size='small')), axis = 1)

print('Done.')

Done.
